In [ ]:
%pip install llama-index-readers-file pymupdf
%pip install llama-index-llms-openai
%pip install llama-index-retrievers-bm25
%pip install llama-index

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### RAG기반 커스텀 검색 엔진 결과 기반 답변 생성하기

### 1. 커스텀 검색 엔진 만들기

##### 1-1 llama index, vector store 세팅

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# llm, embed model 세팅
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

os.environ['OPENAI_API_KEY'] =  # OPENAI API KEY

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
embed_model = OpenAIEmbedding(
    model="text-embedding-3-small", embed_batch_size=256
)

In [ ]:
# 문서 load, 삼성전자 사업보고서 사용
from llama_index.core import SimpleDirectoryReader

folder_path = '/content/drive/MyDrive/데이터/docs/사업보고서'
documents = SimpleDirectoryReader(folder_path).load_data()

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter], embed_model=embed_model
)

##### 1-2 쿼리 rewriting 모델 만들기

In [ ]:
from llama_index.core import PromptTemplate

query_string = "현재 삼성전자의 최대 주주는 누구인가요?"


query_rewriteing_prompt = """
너는 주어진 질문에 대한 정보를 찾을 수 있도록 여러개의 질문를 생성하는 인공지능이야.
주어진 질문를 참고하여서 주어진 질문에 대한 정보를 찾을 수 있는 {num_generate_query} 개의 새로운 질문를 만들어줘.
한 줄에 하나의 질문만 작성 해줘. 번호나 다른 인덱싱은 하지 말아줘.
질문: {query}
새로운 쿼리:
"""

query_gen_prompt = PromptTemplate(query_rewriteing_prompt)

In [ ]:
def generate_queries(llm, query_str, num_generate_query=5):
    prompt = query_gen_prompt.format(
        num_generate_query=num_generate_query - 1, query=query_str
    )
    response = llm.complete(prompt)
    queries = response.text.split("\n")
    answer = [query_str] + queries
    return answer

In [ ]:
queries = generate_queries(llm, query_string, num_generate_query=5)
print(queries)

['현재 삼성전자의 최대 주주는 누구인가요?', '삼성전자의 최대 주주는 몇 주를 보유하고 있나요?', '삼성전자의 최대 주주는 언제부터 해당 지위를 보유하고 있나요?', '삼성전자의 최대 주주는 어떤 이유로 해당 지위를 보유하고 있나요?', '삼성전자의 최대 주주는 다른 기업과의 관련이 있나요?']


##### 1-3 각 쿼리에 대한 정보를 문서로부터 가져오기

In [ ]:
from tqdm.asyncio import tqdm

async def run_queries(queries, retrievers):
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append(retriever.aretrieve(query))

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for query, query_result in zip(queries, task_results):
        results_dict[query] = query_result

    return results_dict

In [ ]:
from llama_index.retrievers.bm25 import BM25Retriever

vector_retriever = index.as_retriever(similarity_top_k=2)

bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=3
)

In [ ]:
results_dict = await run_queries(queries, [vector_retriever, bm25_retriever])

100%|██████████| 10/10 [00:00<00:00, 15.20it/s]


In [ ]:
print(results_dict.keys())

dict_keys(['현재 삼성전자의 최대 주주는 누구인가요?', '삼성전자의 최대 주주는 몇 주를 보유하고 있나요?', '삼성전자의 최대 주주는 언제부터 해당 지위를 보유하고 있나요?', '삼성전자의 최대 주주는 어떤 이유로 해당 지위를 보유하고 있나요?', '삼성전자의 최대 주주는 다른 기업과의 관련이 있나요?'])


In [ ]:
print(results_dict['현재 삼성전자의 최대 주주는 누구인가요?'][0])

Node ID: 8f959081-d1a5-49bf-97f6-636fc58adecc
Text: 다. 최대주주의 최대주주의 개요   (1) 최대주주의 최대주주(삼성물산㈜)의 기본정보    1) 법적ㆍ상업적 명칭:
삼성물산㈜ (Samsung C&T Corporation)    2) 설립일자 및 존속기간     1963년 12월 23일에
동화부동산주식회사로 시작하여, 2014년 7월 제일모직     주식회사로 사명을 변경하였습니다. 또한 1938년 3월 1일
설립된 삼성상회를     모태로 1951년 1월에 설립된 삼성물산주식회사와의 합병을 통하여 2015년     9월
2일(합병등기일) 삼성물산주식회사로 사명을 변경하였습니다.    3) 본사의 주소, 전화번호 및 홈페이지      주소:
서울특별시 강동구 상일...
Score:  0.596



In [ ]:
print(results_dict['삼성전자의 최대 주주는 언제부터 해당 지위를 보유하고 있나요?'][0].get_content())

다. 최대주주의 최대주주의 개요
 
(1) 최대주주의 최대주주(삼성물산㈜)의 기본정보 
 
1) 법적ㆍ상업적 명칭: 삼성물산㈜ (Samsung C&T Corporation) 
 
2) 설립일자 및 존속기간
    1963년 12월 23일에 동화부동산주식회사로 시작하여, 2014년 7월 제일모직
    주식회사로 사명을 변경하였습니다. 또한 1938년 3월 1일 설립된 삼성상회를
    모태로 1951년 1월에 설립된 삼성물산주식회사와의 합병을 통하여 2015년
    9월 2일(합병등기일) 삼성물산주식회사로 사명을 변경하였습니다. 
 
3) 본사의 주소, 전화번호 및 홈페이지 
    주소: 서울특별시 강동구 상일로6길 26(상일동) 
    전화번호: 02-2145-5114 
    홈페이지: www.samsungcnt.com
 
4) 최대주주의 최대주주(삼성물산㈜)의 대표이사ㆍ업무집행자ㆍ최대주주 
(단위 : 명, %)
명 칭출자자수
(명)대표이사
(대표조합원)업무집행자
(업무집행조합원)최대주주
(최대출자자)
성명 지분(%) 성명 지분(%) 성명 지분(%)
삼성물산㈜ 146,861고정석 0.00 - -이재용 18.26
오세철 0.00 - - - -
정해린 0.00 - - - -
※ 2023년 12월 31일, 보통주 기준입니다.
전자공시시스템 dart.fss.or.kr Page 356


##### 1-4 각 쿼리의 결과를 합치기

In [ ]:
from typing import List
from llama_index.core.schema import NodeWithScore

def merge_results_by_sim_score(result_dict, similarity_top_k=5):
  merge_scores = {}
  text2node = {}
  for nodes_with_scores in result_dict.values():
    sorted_nodes_with_scores = sorted(nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True)
    for node_with_score in sorted_nodes_with_scores:
      text = node_with_score.node.get_content()
      text2node[text] = node_with_score
      if text not in merge_scores:
        merge_scores[text] = node_with_score.get_score()

  reranked_results = dict(
      sorted(merge_scores.items(), key=lambda x: x[1], reverse=True)
  )

  reranked_nodes: List[NodeWithScore] = []
  for text, score in reranked_results.items():
      reranked_nodes.append(text2node[text])
      reranked_nodes[-1].score = score

  return reranked_nodes[:similarity_top_k]




In [ ]:
final_results = merge_results_by_sim_score(results_dict)

In [ ]:
for n in final_results:
    print(n.score, "\n", n.text, "\n********\n")

0.5957421303930795 
 다. 최대주주의 최대주주의 개요
 
(1) 최대주주의 최대주주(삼성물산㈜)의 기본정보 
 
1) 법적ㆍ상업적 명칭: 삼성물산㈜ (Samsung C&T Corporation) 
 
2) 설립일자 및 존속기간
    1963년 12월 23일에 동화부동산주식회사로 시작하여, 2014년 7월 제일모직
    주식회사로 사명을 변경하였습니다. 또한 1938년 3월 1일 설립된 삼성상회를
    모태로 1951년 1월에 설립된 삼성물산주식회사와의 합병을 통하여 2015년
    9월 2일(합병등기일) 삼성물산주식회사로 사명을 변경하였습니다. 
 
3) 본사의 주소, 전화번호 및 홈페이지 
    주소: 서울특별시 강동구 상일로6길 26(상일동) 
    전화번호: 02-2145-5114 
    홈페이지: www.samsungcnt.com
 
4) 최대주주의 최대주주(삼성물산㈜)의 대표이사ㆍ업무집행자ㆍ최대주주 
(단위 : 명, %)
명 칭출자자수
(명)대표이사
(대표조합원)업무집행자
(업무집행조합원)최대주주
(최대출자자)
성명 지분(%) 성명 지분(%) 성명 지분(%)
삼성물산㈜ 146,861고정석 0.00 - -이재용 18.26
오세철 0.00 - - - -
정해린 0.00 - - - -
※ 2023년 12월 31일, 보통주 기준입니다.
전자공시시스템 dart.fss.or.kr Page 356 
********

0.476911473327775 
 다. 최대주주의 변동 
 
2021년 4월 29일에 기존 최대주주가 소유하던 당사 주식의 상속으로 인해 최대주주가 삼성
생명보험㈜으로 변동되었습니다. 
 
☞ 최대주주 관련 자세한 사항은 'VII. 주주에 관한 사항'을 참고하시기 바랍니다.
 
라. 상호의 변경 
 
2019년 중에는 종속기업 Samsung Electronics Greece S.A. (SEGR) 사명이 Samsung
Electronics Greece S.M.S.A (SEGR)로 변경되었습니다.
 
2021년 중에는 종속기

### 2. custom search engine을 사용하여 답변 생성하기


In [ ]:
async def get_related_docs(query_string, num_generate_query=3, similarity_top_k=3):
  queries = generate_queries(llm, query_string, num_generate_query=num_generate_query)

  tasks = []
  retrievers = [vector_retriever, bm25_retriever]
  for query in queries:
      for i, retriever in enumerate(retrievers):
          tasks.append(retriever.aretrieve(query))

  task_results = await tqdm.gather(*tasks)

  results_dict = {}
  for query, query_result in zip(queries, task_results):
      results_dict[query] = query_result

  final_results = merge_results_by_sim_score(results_dict, similarity_top_k=similarity_top_k)

  related_docs = ""
  for i, n in enumerate(final_results):
    related_docs += f"""
    [{i}]: {n.text}
    """
  return related_docs

docs = await get_related_docs(query_string="삼성전자의 DX부문 매출액은 얼마인가요?")
print(docs)

100%|██████████| 6/6 [00:00<00:00, 13.43it/s]


    [0]: 나. 영업실적 
 
제55기 당사 매출액은 메모리 등 부품 사업의 약세로 전년 대비 43조 2,959억원(14.3%) 감
소한 258조 9,355억원을 기록하였으며, 영업이익 또한 전년 대비 36조 8,097억원(84.9%)
감소한 6조 5,670억원을 기록하였습니다. 법인세비용차감전순이익은 전년 대비 35조
4,342억원(76.3%) 감소한 11조 63억원, 당기순이익은 전년 대비 40조 1,670억원(72.2%)
감소한 15조 4,871억원을 기록하였습니다.
 
수익성 지표 측면에서는 ROE가 전년 대비 12.6%p 감소한 4.3%, 매출액순이익률이 전년 대
비 12.4%p 감소한 6.0%이나, 지속적으로 견실한 재무구조를 유지할 수 있도록 노력하고 있
습니다.
    
    [1]: 등
기타 - 65,503 84.0%
소 계 77,933 100.0%
기타 - - 462 -
총  계 1,000,548 -
※ 매입액은 부문 등 간의 내부거래를 포함하고 있지 않습니다.
※ 비중은 각 부문의 원재료 총매입액 대비 각 품목의 매입액 비중입니다.
※ 주요 매입처 중 삼성전기㈜는 계열회사입니다.
※ AP와 그 주변부품의 높은 연관성을 감안하여, 2023년부터 기존 모바일AP에서 모바일AP 솔루션(AP와 AP전용 IC류 자
재 포함)
    가격으로, DX부문 주요 원재료 현황을 작성하였습니다.
매입액(억원) 2022년 2021년
모바일AP 솔루션 113,790 76,295
전자공시시스템 dart.fss.or.kr Page 28
    
    [2]: 2. 주요 제품 및 서비스
 
가. 주요 제품 매출
 
당사는 TV, 냉장고, 세탁기, 에어컨, 스마트폰 등 완제품과 DRAM, NAND Flash, 모바일AP
등 반도체 부품 및 스마트폰용 OLED 패널 등을 생산ㆍ판매하고 있습니다. 아울러
Harman을 통해 디지털 콕핏, 카오디오, 포터블 스피커 등을 생산ㆍ판매하고 있습니다.
 
2023년 매출은 DX 부문이 169조 9,923억원(65.7%),

In [ ]:
from llama_index.llms.openai import OpenAI

os.environ['OPENAI_API_KEY'] =  # OPENAI API KEY

async def answer(query):
  docs = await get_related_docs(query_string=query)
  prompt = f"""
  주어진 문서를 활용하여 사용자의 질문에 대해 친절하게 답변해줘

  문서
  {docs}

  질문: {query}
  """
  response = OpenAI().complete(prompt)
  return response.text

In [ ]:
query_list = ["현재 삼성전자의 최대 주주는 누구인가요?", "삼성전자의 DX부문 매출액은 얼마인가요?"]

for q in query_list:
  print(f"Q: {q}")
  print(f"A: {await answer(q)}")
  print("\n=========================\n")


Q: 현재 삼성전자의 최대 주주는 누구인가요?


100%|██████████| 6/6 [00:00<00:00, 13.41it/s]


A: 현재 삼성전자의 최대 주주는 이 문서에서 언급되지 않았습니다. 삼성전자의 최대 주주 정보를 확인하려면 해당 기업의 공시나 보고서를 참고하시기 바랍니다.


Q: 삼성전자의 DX부문 매출액은 얼마인가요?


100%|██████████| 6/6 [00:00<00:00,  8.75it/s]


A: 삼성전자의 DX부문 매출액은 169조 9,923억원입니다.




##### context 늘리기

In [ ]:
from llama_index.llms.openai import OpenAI

os.environ['OPENAI_API_KEY'] =  # OPENAI API KEY

async def answer(query):
  docs = await get_related_docs(query_string=query, num_generate_query=3, similarity_top_k=10)
  prompt = f"""
  주어진 문서를 활용하여 사용자의 질문에 대해 친절하게 답변해줘

  문서
  {docs}

  질문: {query}
  """
  response = OpenAI().complete(prompt)
  return response.text

In [ ]:
query_list = ["현재 삼성전자의 최대 주주는 누구인가요?", "삼성전자의 DX부문 매출액은 얼마인가요?"]

for q in query_list:
  print(f"Q: {q}")
  print(f"A: {await answer(q)}")
  print("\n=========================\n")

Q: 현재 삼성전자의 최대 주주는 누구인가요?


100%|██████████| 6/6 [00:00<00:00, 13.45it/s]


A: 현재 삼성전자의 최대 주주는 삼성물산㈜이며, 최대주주인 삼성물산㈜의 대표이사는 이재용씨이고 지분은 18.26%입니다.


Q: 삼성전자의 DX부문 매출액은 얼마인가요?


100%|██████████| 6/6 [00:00<00:00, 13.51it/s]


A: 답변: 삼성전자의 DX부문 매출액은 169조 9,923억원입니다.


